In [52]:
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import (
    PdfPipelineOptions,
    TesseractCliOcrOptions,
)


import re
from rag import *
from docling.document_converter import DocumentConverter, PdfFormatOption
from transformers import AutoTokenizer
hf_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
from docling.chunking import HybridChunker

In [14]:

source = r"C:\Users\tallar\Downloads\Famille bamendjou de liege-20250801T201343Z-1-001\Famille bamendjou de liege\ARESBAL_ ROI et statut.pdf"  # Chemin local ou URL

In [39]:

def x(source):
    pipeline_options = PdfPipelineOptions()
    pipeline_options.do_ocr = True
    converter = DocumentConverter(
        format_options={
            InputFormat.PDF: PdfFormatOption(
                pipeline_options=pipeline_options,
            )
        }
    )
    result = converter.convert(source)
    doc=result.document
    #all_texts = [elem.text for elem in doc.iter_text()]
    #full_text = " ".join(all_texts)
    return doc

In [40]:
result=x(source)

C:\Users\tallar\Documents\PROJETS\GenAI\venv\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [28]:
print(len(result))

C:\Users\tallar\Documents\PROJETS\GenAI\venv\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TypeError: object of type 'DoclingDocument' has no len()

In [48]:
docs = result.export_to_text()
print(docs)

Parameter `strict_text` has been deprecated and will be ignored.


## 25 MARS 2023 A.RES.BA.L.

Rue de la briqueterie 38 - 4621 Fléron

## A.RES.BA.L.

Association des ressortissants Bamendjous de Liège et environs

## Table

## STATUT &amp; REGLEMENT D'ORDRE INTERIEUR

## des matières

## I. 2

But et objectif

Art 1er : L'association est à but non lucratif, apolitique et est dénommée A.RES.BA.L

Art 2 : LES ASSISES

Art 3 : DES MEMBRES.

∙

2

∙

2

∙

2

Art 4 : Du bureau de la réunion.

3

## II. 3

3

5

5

5

5

5

6

6

6

8

8

| Art 5 : Des attributions des                                                 | membres du bureau.   |   3 |
|------------------------------------------------------------------------------|----------------------|-----|
| Art 6 : Des ressources de la réunion                                         |                      |   5 |
| Art 7 : De l'élection des membres du bureau                                  |                      |   5 |
| Art 8 : des conditions d'éligibilité pour Président -Commissaire aux comptes | -Tré

In [49]:
text = re.sub(r'-\n', '', docs)
text = re.sub(r'\n', '', text)
raw_paragraphs = text.split('.')
print([p.strip().replace('\n', ' ') for p in raw_paragraphs if len(p.strip()) > 40])

['Rue de la briqueterie 38 - 4621 Fléron## A', "Association des ressortissants Bamendjous de Liège et environs## Table## STATUT &amp; REGLEMENT D'ORDRE INTERIEUR## des matières## I", "2But et objectifArt 1er : L'association est à but non lucratif, apolitique et est dénommée A", '335555566688| Art 5 : Des attributions des                                                 | membres du bureau', "|   3 ||------------------------------------------------------------------------------|----------------------|-----|| Art 6 : Des ressources de la réunion                                         |                      |   5 || Art 7 : De l'élection des membres du bureau                                  |                      |   5 || Art 8 : des conditions d'éligibilité pour Président -Commissaire aux comptes | -Trésorier           |   5 || Art 9 : Du mandat du bureau                                                  |                      |   5 || Art 11 : Des finances                               

In [43]:
from pdfminer.high_level import extract_text
texte = extract_text(source)

In [44]:
print(texte)

25 MARS 2023 
A.RES.BA.L. 
Rue de la briqueterie 38 – 4621 Fléron 

A.RES.BA.L. 

Association des ressortissants Bamendjous de Liège et environs 

                                                                                                                                 Rev0 – version 03/2023 

 
 
 
STATUT & REGLEMENT D’ORDRE INTERIEUR 

Table 
des matières 

I. 

2 

But et objectif 

Art 1er : L’association est à but non lucratif, apolitique et est dénommée A.RES.BA.L 

Art 2 : LES ASSISES 

Art 3 : DES MEMBRES. 

∙ 

∙ 

∙ 

2 

2 

2 

Art 4 : Du bureau de la réunion. 

3 

II.  3 

Art 5 : Des attributions des 

Art 6 : Des ressources de la réunion

A.S.B.L. 

membres du bureau. 

Art 7 : De l’élection des membres du bureau 

Art 8 : des conditions d’éligibilité pour Président-Commissaire aux comptes -Trésorier 

Art 9 : Du mandat du bureau 

Art 11 : Des finances 

Art 12. Du non-paiement d’une sanction 

Art 13. : des droits et devoirs des membres 

Art.14 : Les aides 

A

In [3]:
from llm import *
model = model("mistral")
question="quels sont les types de membres"
contexte = "il existe 3 types de membre: les membres honoraires, les affiliés, et les sympathisants"
model.set_context(contexte)
model.set_task(question)
resultat = model.ask(question,"mistral")
print(resultat)

200
True
Les types de membres dans le contexte d'une société, organisation ou association peuvent comprendre :

1. Membres actifs : Ces membres prennent part aux activités et décisions de l'organisation. Ils ont généralement le pouvoir de voter et de participer à des élections.

2. Membres passifs : Ceux-ci bénéficient des avantages d'être membres mais n'ont pas les droits de vote ou ne participent pas aux activités décisionnelles. Ils peuvent recevoir des informations et des actualités, et certains plans ou offres exclusivement destinés aux membres.

3. Membres honoraire : Ce sont des personnes qui ont apporté une contribution exceptionnelle à l'organisation dans le passé mais ne sont plus impliquées de manière active. Ils peuvent recevoir un titre honorifique ou certains autres avantages spécifiques.

4. Membres émérite : Cela peut être un membre qui a atteint un certain âge, retiré de l'activité ou reçu une reconnaissance pour son travail. Les droits et avantages varient d'une organ